<h1 style="color:black" align="center">Линейная классификация</h1>

Мы помним что функционал ошибки выглядел следующим образом:

$\large Q(a, X) = \frac{1}{\ell} \sum\limits_{i=1}^{\ell} [a(x_i) = y_i]$ - долей правильных ответов (accuracy)

Нам будет удобнее решать задачу минимизации, поэтому будем вместо этого использовать долю неправильных ответов:

$\large Q(a, X) = \frac{1}{\ell} \sum\limits_{i=1}^{\ell} [a(x_i) \ne y_i]$ - доля ошибок.

Подставили
Так как этот функционал не дифференцируемый, то его необходимо заменить на функцию, которая оценивает сверху идикатор ошибки. Записали оценку сверху мы следующим образом:

$\large 0 \le \frac{1}{\ell} \sum\limits_{i=1}^{\ell} [y_i \langle w, x \rangle < 0] 
\le 
 \frac{1}{\ell} \sum\limits_{i=1}^{\ell} \tilde L (y_i \langle w, x \rangle) \to \underset{w}{\text{min}}$

<h1 style="color:#008B8B">1. Логистическая регрессия</h1>

<h2 style="color:#008B8B">1.1 Оценивание вероятностей</h2>

### Для чего необходимо оценивать вероятность?

Предположим, имеется модель $\text{b(x)} \in [0, 1]$. Представим, что модель выдала на некотором объекте уверенность равную $0.8$ в классе +1. Как это можно интерпретировать?

Например, имеется множество из 10 объектов, где модель возвращает приблизительно одну веренность в своём прогнозе, тогда мы ожидаем, что среди этих объектов 80% будут положительными.

Предположим, мы построили модель, кторая оценивает вероятность положительного класса. Записывать это будем следующим образом:

$\large b(x) \approx p(y = +1 | x)$ - модель оценивает то, что класс положительный для данного объекта $x$. Оценивание веротяности принадлежности объекта к некоторому классу позволяет посмотреть уверенность модели в своём прогнозе. Также бывают задачи, когда нет понимания того, какой ответ в задачи: 

### Пример 1

Пример, мы решаем задачу предсказания кликов пользователя по рекламе, где необходимо предсказать кликнет ли пользователь по данному рекламному банеру. Имеются следующие 3 категориальных признака по которым мы предсказываем нажмёт ли пользователь на рекламный банер.

Так как объект описывается тремя идентификаторами, то может быть ситуация, когда пользователь отправлят один и тот же запрос на который показывается один и тот же банер:

|id пользователя | id запроса | id банера | Нажал ли пользователь? |
|:--------------:|:----------:|:---------:|:----------------------:|
|10              |50          |527        |0                       |
|10              |50          |527        |0                       |
|10              |50          |527        |0                       |
|10              |50          |527        |0                       |
|10              |50          |527        |1                       |

Пользователь может в большинстве случаев не нажимать на рекламу, спустся определённое количество запросов он кликнул на данный банер. Проблема заключается в том что один объект встречается много раз с рзными целевыми переменным (большенство раз 0). Если данную обучающую выборку передать классификатору, который возвращает только класс объекта, тогда ему будет тяжело извлечь закономерность из этой выборки, так как объект один, а ответы целевой переменной разыне. Тогда, можно сделать так, чтобы классификатор возвращал не класс, а вероятность пренадлежность объекта к некоторому классу.

### Пример 2

Представим, что пользователь вводит некоторый запрос, на который необходим показать один из 4 банера. За каждый кликнутый банер мы получаем определённую сумму денег, обазначим как $c(x)$. Показывать самый дорогой банер будет плохой идей, так как пользователь может на него не нажать, для этого необходимо ввести вероятность клика $p(y = +1| x)$ которую оценивает модель.

Что мы максимизируем в данной задаче? Необходимо максимизировать математическое ожидание прибыли, а что такое мат.ожидание прибыли? Это $с(x) \cdot p(y = +1| x)$

|$$c(x)$$|$$p(y = +1| x)$$ |$$с(x) \cdot p(y = +1| x)$$ |
|:------:|:---------------:|:--------------------------:|
|1       |0.5              |0.5                         |
|5       |0.1              |0.5                         |
|10      |0.3              |3                           |
|100     |0.01             |1                           |

Лучшее математическое ожидание будет $3$. Значит данный банер лучше всего показывать. Соответсвенно, данная вероятность помогает выбрать то, что необходимо показывтаь пользователю.

### Пример 3

Банк принимает решение того, какой продукт необходимо предложить пользователю. Имеется продукт мобильный банк, который он подключит с вероятностью 0.9, либо мы можем предложить пользоватлею кредит, на который он согласится с вероятностью 0.1. Если предлогать пользователю продук по вероятности, тогда мы не сможем узнать цену рекомендации. Если на основе вероятности оценить то, сколько мы заработаем, тогда предложить крдеит пользвателю будет более хорошей идеей, так как прибыль намного будет больше.

### Что такое оценивание вероятностей?

Предположим, имется набор объектов $\{x_1, \ldots, x_n\}$, который устроент так, что модель вовращает одинаковы числа на этих объетках $b(x_1) = b(x_2) = \ldots = b(x_n)$, а целивая переменная на данных объектах имеет значения $\{y_1, \ldots, y_n\}$.

Мы хотим, чтобы модель выдала на этих объектах долю положительных среди них. Это требование можно записать через функцию потерь:

$\large \underset{b \in \mathbb{R}}{\text{argmin}} \frac{1}{n} \sum\limits_{i=1}^n L(y_i, b)$ - ищем такое $b$ из всех возможных, которое даёт минимальную среднюю ошибку на $n$ объектах.

И мы хотим, чтобы данная константа совпадала с долей положительных объектов:

$\large \underset{b \in \mathbb{R}}{\text{argmin}} \frac{1}{n} \sum\limits_{i=1}^n L(y_i, b) \approx \frac{1}{n} \sum\limits_{i=1}^n [y_i = +1]$

Стоит подметить, что модель выдаёт один и тот же прогноз на множестве $\{x_1, \ldots, x_n\}$ объектов, мощность которого $n$ (это не все возможные объекты).

Запишем более точную формулировку, так как мы работаем с вероятностью. Для этого будем считать, что $n \to \infty$, другими словами, имеется некоторая область (множество) из всех возможных объектов, где бесконечное количество объектов и модель на них возвращает одинаковый прогноз. Тогда посчитаем математическое ожидание функции потерь на объекте $x$ и мы требуем, чтобы $\text{argmin}$ данного математического ожидания достигался на вероятности положительного класса для данного объетка $p(y = +1 | x)$

$\large \underset{b \in \mathbb{R}}{\text{argmin}} =\mathbb{E} (L(y, b)|x) = p(y = +1 | x)$

**Переход от первой формулы к второй**

В первой формуле предпологается, что есть конечное множество объектов, которых $n$ штук. Функция потерь устроена так, что если мы будем искать оптимальную константу $b$ для прогноза на $n$ объектах, тогда оптимальное значение $b$ должно быть примерно равно доле положительных объектов в этом множестве. Значит, мы требуем для множества объектво где модель выдаёт один и тот же прогноз, чтобы этот прогноз был приерно равен вероятности положительного класса.

Переход к второй формуле. Мы говорим, что $n$ теперь стремится к бесконечности. Имеется один объект $x$, экземпляров которого большое количество, а именно $n$ раз данный объект встречается в выборке и на этом объекте разные значения целевой переменной. Тогда, мы считаем математическое ожидание функции потреть по распределению $y$ в данной точке и минимальное математическое ожидание должно достигаться при константе $b$ равной вероятности положительного класса.

$b$ оценивает вероятность положительного класса, поэтому $b$ должно аппроксимировать веротяность положительного класса. 

(не уверне в этом предложении) Почему у нас есть $n$ объектов, где прогноз одинаковый? Так как это один объект, значит и прогноз и целевая переменная будут иметь одно значение. Такую ситуацию мы уже разобрали выше, поэтому формула один была переписана на вероятностный язык.

**Про математическое ожидание**

Математическое ожидание берётся по распределению $y$ при условии данного $x$ $p(y|x), где $$y \in \{-1, +1\}$. Математическое ожидание будет иметь следующий смысл:

Это вероятность того, что $y$ равен +1 при условии $x$...

$\large \mathbb{E} = p(y=+1 |x) \cdot L(+1, b) + p(y = -1 | x) \cdot L(-1, b)$


Почему записали математическое ожидание? Так как мы $n$ устремляем в бесконечность, тогда вместо средней ошибки можно записать математическое ожидание ошибки.  А дальше, по закнону больших чисел в первой формуле, доля положительных объектов будет стремится к вероятности положительного класса. Ну и мы дальше во второй формуле требуем строгое равенство, чтобы функция потерь была устроена так, что для точки с бесконечным количеством объектов оптимально выдавать прогноз равный вероятности положитлеьного класса.

**Дополнительные вопросы**

Что означает запись при условии $x$? Это означает, что мы рассматриваем распределение ответов конкретно для данного объекта. Допусим, имется объект $x_1$ на котором вероятность положитлеьного класса $p(y = +1 |x) = 0.9$, есть другой объект $x_2$, где вероятность равна $p(y = +1 |x) = 0.4$. Получается, что это вероятность того, что данный объект $x$ положительный.

Что означает распределение ответов для одного объекта? Мы значем что объект один, но этот объект может встречаться несколько раз, это мы рассаматривали выше. Предположи, есть пространство объектов, где много положителных и среди них в середине расположен один отрицательный, тогда вероятнотность того, что этот объект будет отрицательным в данной области (множестве) будет небольшой. Поскольку эти объекты расположены рядом и модель вовзращает практически идентичный прогноз, тогда мы ожидаем, что вероятность того, что этот объект отрицательный будет небольшой. 

Под объекта расположенными рядом понимается то, что объекты с практически одинаковым признаковым описанием, которые группируются в признаковом простарнстве (мы это называли множеством), тогда логично,что скорее всего они относятся к одной группе. Соответсвенно записи формул по сумме $n$ объектов понимается что, имеется некая группа мощностью $n$, где их признаковое описание практически похоже, значит и модель будет выдавать приблизительно один резултат.

### Пример 4

Применим требование для квадратичной ошибки.

$\large L(y, b) = (y-b)^2$

Посмотрим, что будет оптимальное для такой ошибки, если у нас имеется много объектов с одинаковым признаковым описанием:

$\large \mathbb{E} ((y - b)^2 | x) = p(y = +1 |x) \cdot (1 - b)^2 + p(y = 0|x) \cdot (-b)^2$

Для того, чтобы требование выполнялось для $MSE$, необходимо пронумировать классы $y \in \{0, 1\}$. 

$p(y = +1 |x) \cdot (1 - b)^2$ - вероятность того, что класс положительный;

$p(y = 0|x) \cdot (-b)^2$ - вероятность того, что класс отрицательный.

Найдём минимум математического ожидания:

$\large \frac{\partial}{\partial b} = 2 \cdot p(y = +1 |x) (b-1) + 2p(y=0|x)b = $

Заметим, что $p(y=0|x) = (1-p(y = +1|x))$. Для краткости заменим $p(y = +1|x) = p$.

$\large = 2pb -2p + 2b - 2pb = 0$

Получаем, что $\large b = p(y = +1|x)$ - оптимально для модели (константы) быть равной вероятности положительного класса. 

**Как это можно интерпретировать?** Что такое вероятность положительного класса для данного объетка? Этооля положительных экземпляров среди объектов, если экземпляров данного объетка много. После эту долю мы умнжаем на ошибку в случае, если прогноз $b$ на данный экземплярах. Дальше долю отрицательных положительных экземпляров умножаем на ошибку при прогнозе $b$ для отрицатлеьных объектов и смотрим оптимальное значение $b$, оказыватся что опитмально брать $b$ в качестве вероятности положительного класса. Получается, если мы рассматриваем среднеквадратичную ошибку, тогда она устроена так, что для ней оптимально выдавать в некоторой области долю положитлеьных объектов.

### Пример 5

Если првоерить данное свойство для абсолютной ошибки:

$\large L(y, b) = |y - b|$

Опитмальное $b\in \{0, 1\}$

### Что мы рассмотрели выше?
Пусть выполнено следующее свойство:

$\large \underset{b \in \mathbb{R}}{\text{argmin}} =\mathbb{E} (L(y, b)|x) = p(y = +1 | x)$

1. Если один объект $x$ встречается много раз в выборке и нет других объектов  в выборке, тогда модель $b(x)$ будет оценивать вероятность положительного класса. (такуой случай не встречается на практике, он теоретический)

2. Если много разных объектов в выборке и нет ограничений на прогноз в каждом из них (модель может быть любой функцией, где на каждом объекте она может вернуть любое чилсо), тогда нет гарантий что данное свойство будет выполняться. 

3. Если много разных объектов в выборке и прогозны по объектам ограничиваются видном модели. Это означает, например, если некоторая метрика расстояния между двумя объектами небольшая, тогда выходы объектов примерно равны на этих двух объектах:

    $\large p(x_1, x_2) < \epsilon \Longrightarrow b(x_1) \approx b(x_2)$
    
    Из-за того, что выходы модели связаны на объекта, тогда свойство будет выдавать приблизительно тоже самое. Хотя точно похожих объектов нет, но есть объекты, на которых выходы можеди похожи, тогда изходя из свойства, мы будем примено оценивать их вероятность.

### Функционал для квадрата ошибки

Мы проверили, что квадарт ошибки $\large L(y - b)^2$ обладает нужным свойством, поэтому его можно использовать для обучения классификатора который оцениват уверенность. Толко $b \in \{0, 1\}$, если взять $b(x) = \langle w, x \rangle$ равное скалярному произведению, тогда не сходистя, ведь скалярное произведение возвращает любое вещественное число, для решения это проблемы можно придумать преобразование скалярного приозведения чтобы оно переводило вещественно число в отрезок от 0 до 1. Мы будем использовать для этого сигмойду:

$\large \sigma (z) = \frac{1}{1 + e^{-z}}$

Тогда обучать линейный классификатор будем на следующем функционале:

$\large \frac{1}{\ell} \sum\limits_{i=1}^{\ell} (\sigma (\langle w, x_i \rangle) - y_i)^2 \to \underset{w}{\text{min}}$

Вот мы записали модель, которая будет оценивать верояность.

**Чем плохой этот функционал?**

Сигмойда быстро достигает 0 или 1 и становится практически константой. Предположим что на объекте  $x_1$, где целевая переменная равана $y=1$ веса имеют такое значение, что их скалярное произведение даст $\langle w, x_1 \rangle = -1000$, тогда $\sigma(\langle w, x_1 \rangle) \approx 0$. Ну и видим, что ошибка получается огромной.

Допустим мы используем стохастический градиентый спуск и считаем градинт функционала по $w$:

$\large \nabla_w (\sigma(\langle w, x_1 \rangle) - y_1)^2= (\sigma(\langle w, x_1 \rangle) - y_1)^2 \cdot \sigma'(\langle w, x_1 \rangle) \cdot x_1$

Здесь мы видим производную сигмойды $\sigma'(\langle w, x_1 \rangle)$ в точке примеро равной $-1000$, а чему будет равна производна в этой точке если посмотреть на график? Конечно же примерно нулую, тогда и градиент тоже будет примерно равен нулю.

Получается, если мы ошибаемся на объекте и попадаем константную зону сигмойды, тогда градиентый спуск не сможет оттуда выйти, это называется затуханием градиента. Поэтому использовать $MSE$ для обучения такой модели будет плохой идеей.

<h2 style="color:#008B8B">1.2 Правдоподобие и логистические потери</h2>

Имеются объекты $x_1, \ldots, x_{\ell}$ с правильными ответами $y_1, \ldots, y_{\ell}$, где $y \in \{-1, +1\}$. А модель оценивает веротяность положительного класса $b(x_1), \ldots, b(x_{\ell})$. Для каждой точки известна веротяность того, что этот объект положительный (с точки зрения модели). Тогда запишем правдоподобие, которое говорит о том, насколько модель согласуется с данными:

$\large Q(a, X) = \prod\limits_{i = 1}^{\ell} b(x_i)^{[y_i = +1]} (1 - b(x_i))^{[y_i = -1]} \to \underset{w}{\text{max}}$

Мы проходимся по всем объектам, если объект положителен, тогда максимизируем $b(x_i)^{[y_i = +1]}$, то есть вероятность того, что это положительный класс должна быть близка к еденице, так как целевая переменная имеет значение 1. Если объект отрицателен, тогда вероятность положительного класса на этом обоъекте должна быть близка к нулю $b(x_i))^{[y_i = -1]}$.

Правдоподобие будет максимальным, если для каждого положитлеьного объекта $b(x) = 1$, а для каждого отрицательного $b(x) = 0$.

Данное правдоподобие можно использовать как функционал для обучения алгоритма, с той лишь оговоркой, что удобнее оптимизировать его логарифм, который позволяет убрать произведение чисел от 0 до 1, а миниус нужен для замены максимизации на минимизацию:

$\large -\sum\limits_{i = 1}^{\ell} \left( [y_i = +1] \log b(x_i) + [y_i = -1] \log (1 - b(x_i)) \right) \to \underset{w}{\text{min}}$

Функция потреь записывается следующим образом:

$\large L(y, b) = -[y_i = +1] \log b - [y = -1] \log (1-b)$ - log-loss

Данная функция потерь называется логарифмической (log-loss). Покажем, что она также позволяет корректно предсказывать вероятности. Запишем матожидание функции потерь в точке $x$:

$\large \mathbb{E} \biggl[L(y, b) | x \biggr] = \mathbb{E} \biggl[-[y = +1] \log b - [y = -1] \log(1 - b) | x \biggr] =$
$\large -p(y = +1 | x) \log b - (1 - p(y = +1 | x)) \log(1 - b)$

Продифференцируем по $b$:

$\large \frac{\partial}{\partial b} \mathbb{E} \biggl[L(y, b) | x \biggr] =
    -\frac{p(y = +1 | x)}{b} + \frac{1 - p(y = +1 | x)}{1 - b} = 0 \Longrightarrow  b = p(y = +1|x)$

Получаем, что $\large b = p(y = +1|x)$ - оптимально для модели (константы) быть равной вероятности положительного класса. И эту функцию потреть можно уже использовать для оптимизации модели.

<h2 style="color:#008B8B">1.3 Логистическая регрессия</h2>

Тогда $\large b(x) = \sigma(\langle w, x \rangle)$ и подставим это $b(x)$ в log-loss:

$\large -\sum\limits_{i = 1}^{\ell} \left([y_i = +1] \log \frac{1}{1 + \exp(-\langle w, x_i \rangle)} + [y_i = -1] \log \frac{\exp(-\langle w, x_i \rangle)}{1 + \exp(-\langle w, x_i \rangle)} \right) =$

Разделим числитель и знаменатель на числитель, после перевернём дробь:

$\large \log \frac{\exp(-\langle w, x_i \rangle)}{1 + \exp(-\langle w, x_i \rangle)} = \log \frac{1}{1 + \exp(\langle w, x_i \rangle)}= \log 1 + \exp(\langle w, x_i \rangle)$ 

Логарифм дроби это минус логарифм перевёрнутой дроби, тогда внесём минус внутрь:

$\large = \sum\limits_{i = 1}^{\ell} \biggl([y_i = +1] \log (1 + \exp(-\langle w, x_i \rangle)) + [y_i = -1] \log (1 + \exp(\langle w, x_i \rangle) \biggl)=$

Можем заметить, что эти два слогаемых одинаковые, только под экспонентой одного стоит минус, а другого плюс. Мы можем убрать идикатор и записать всё это как единое выражение внеся вутрь экспоненты $-y_i$. Тогда если $y_i$ положителен, то получаем логариф от экспоненты... минус скалряное произведение, как в первом слагаемом, а если $y_i$ отрицательный, тогда то получаем логариф от экспоненты... плюс скалярное произведение как во втором слогаемом. Значит, это две идентичные записи:

$\large = \sum\limits_{i = 1}^{\ell} \log \left(1 + \exp(-y_i \langle w, x_i \rangle) \right)$

И эту функцию потерь мы записывали, которую мы называли логистической. Здесь есть отступ. У этой функции все нормально с градиентом. Этот метод называется логистической регрессией.

In [ ]:
<h1 style="color:#008B8B">2. Метод опорных векторов</h1>

<h2 style="color:#008B8B">1.1 Оценивание вероятностей</h2>